In [1]:
import numpy as np
import pandas as pd
import qrpm_funcs as qf

def gradientmethod(xinit,objfunc,talkative):
    global order_magnitude    #variable used to size objective function around 1
    #Gradient descent method
    #Starting point
    x=xinit
    iteration=0
    #initialize order of magnitude - objfunc uses this to scale objective
    order_magnitude=0.
    
    if talkative:
        print("Initial point:",x[0])
        print("Objective function at initial point:",objfunc(x[0]))
        print("Gradient at initial point:",gradfunc(x[0],objfunc))
    
    #Parameters for backtracking
    alpha = .25
    beta = .75
    epsilon=10**(-10)
    maxiter=10

    #Iterate as long as necessary
    while True:
        deltax=-gradfunc(x[iteration],objfunc)
        delta=1

        #Backtracking
        current_obj=objfunc(x[iteration])
        grad_squared=np.matmul(deltax,deltax)
        if talkative: print(iteration," obj:",current_obj, \
                " grad^2:",grad_squared)
        if grad_squared < epsilon:
            break     #Done

        while True:
            y=x[iteration]+delta*deltax
            new_obj=objfunc(y)
            if new_obj > current_obj-delta*alpha*grad_squared:
                delta*=beta
            else:
                break   #Backtracking done

        if talkative: print("Delta from backtracking:",delta)
        x.append(y)
        iteration+=1
        if iteration > maxiter:
            break
    return(x)

def gradfunc(x, objfunc):
    #Take gradient of function by differencing
    n=len(x)
    epsilon=10**(-8)/n
    bf_x=objfunc(x)

    gradvec=np.array([-bf_x]*n)
    for i in range(n):    #perturb each argument a little
        little_vec=np.zeros(n)
        little_vec[i]=epsilon
        gradvec[i]+=objfunc(np.add(x,little_vec))
    gradvec/=epsilon
    return(gradvec)


In [2]:
#Get 3 currencies until the end of
#previous year. Form sample covariance matrix

lastday=qf.LastYearEnd()
#Swiss franc, pound sterling, Japanese Yen
seriesnames=['DEXSZUS','DEXUSUK','DEXJPUS']
cdates,ratematrix=qf.GetFREDMatrix(seriesnames,enddate=lastday)
multipliers=[-1,1,-1]

lgdates,difflgs=qf.levels_to_log_returns(cdates,ratematrix,multipliers)

#Mean vector and covariance matrix are inputs to efficient frontier calculations
d=np.array(difflgs)*100
m=np.mean(d,axis=0)
c=np.cov(d.T)

In [3]:
def target_func(x, gamma=0):
    temp = np.array([1/3+x[0],1/3+x[1],1/3-x[0]-x[1]])
    exponent = -sum([item*np.log(item) for item in temp])
    temp = temp.reshape(-1,1)
    return 1/2*(temp.T@c@temp)[0][0] - gamma*np.exp(exponent)


In [4]:
# (a)
weights = gradientmethod([[-0.3,0]],target_func,talkative=False)
res = weights[-1]
numerical_res = [1/3+res[0],1/3+res[1],1/3-res[0]-res[1]]
print(numerical_res)

[0.06542006382815135, 0.5026582901668608, 0.4319216460049877]


In [5]:
# compare with the answer from formula (4.6)
c_inv = np.linalg.inv(c)
u = np.ones((3,1))
theoretical = c_inv@u*(1/(u.T@c_inv@u)[0][0])
theoretical_res = [float(i) for i in theoretical]
print(theoretical_res)
print(abs(np.array(numerical_res)-np.array(theoretical_res))<0.01)

[0.05751845131641792, 0.5098898635682844, 0.4325916851152977]
[ True  True  True]


In [6]:
# (b)
def target_func2(x):
    return target_func(x, 100)

weights = gradientmethod([[0,0]],target_func2,talkative=False)
res = weights[-1]
numerical_res = [1/3+res[0],1/3+res[1],1/3-res[0]-res[1]]
print(numerical_res)

[0.33327549168470694, 0.33336585946873026, 0.33335864884656274]


In [7]:
# (c)
gamma = 0.001
while True:
    def target_func3(x):
        return target_func(x, gamma)
    res = gradientmethod([[-0.3,0]],target_func3,talkative=False)[-1]
    if abs(1/3+res[0]-0.2) < 0.001:
        print("Gamma: ", gamma)
        print([1/3+res[0],1/3+res[1],1/3-res[0]-res[1]])
        break
    else:
        gamma += 0.001

Gamma:  0.02000000000000001
[0.19912468620187174, 0.4167829166879334, 0.3840923971101948]
